In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
import torchvision
from torchvision import transforms as T

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision.models as models
root = '/Users/kaiwendai/ML-project/cropped_dataset'

In [2]:
def data_transform():
    transforms = [T.Resize((256,256)),T.ToTensor(),]
    
    return T.Compose(transforms)

In [3]:
dataset = torchvision.datasets.ImageFolder(root, transform = data_transform())
dataset_size = len(dataset)
train_size = int(dataset_size * 0.6)
val_size = int(dataset_size * 0.2)
test_size = dataset_size - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

In [10]:
BATCH_SIZE = 4

train_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)

val_loader = torch.utils.data.DataLoader(val_dataset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)

In [11]:
def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new on
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes)
    return model

In [12]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = models.resnet50(pretrained=True)
# move model to the right device
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [13]:
criterion = nn.CrossEntropyLoss()

In [14]:
def train(model, device, train_loader, optimizer, epoch, log_interval=100):
    model.train() 
    for batch_idx, (data, target) in enumerate(train_loader):   
        data, target = data.to(device), target.to(device)     # use gpu if possible
        optimizer.zero_grad()                                 # clean gradients
        output = model(data)                                  # make a prediction
        loss = criterion(output, target)                      # what is loss

        loss.backward()                                       # calculate loss
        optimizer.step()                                      # update weights (update filters in this case)
        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:0f}%)]\tLoss: {:.6f}".format(
                epoch, 
                batch_idx * len(data), 
                len(train_loader.dataset), #60000
                100. * batch_idx / len(train_loader), 
                loss.item()
            ))


In [15]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data) 
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
                       
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            

            correct += pred.eq(target.view_as(pred)).sum().item()


    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [17]:
train(model, device, train_loader, optimizer, epoch=1,log_interval=100)
test(model, device, test_loader)

Train Epoch: 1 [0/2443 (0.000000%)]	Loss: 0.574998
Train Epoch: 1 [400/2443 (16.366612%)]	Loss: 0.691860
Train Epoch: 1 [800/2443 (32.733224%)]	Loss: 0.393034
Train Epoch: 1 [1200/2443 (49.099836%)]	Loss: 0.661761
Train Epoch: 1 [1600/2443 (65.466448%)]	Loss: 0.039063
Train Epoch: 1 [2000/2443 (81.833061%)]	Loss: 0.131410
Train Epoch: 1 [2400/2443 (98.199673%)]	Loss: 0.002097

Test set: Average loss: 0.3045, Accuracy: 737/815 (90%)

